# leaflet

An exploration of ipyleaflet compatible models for clojupyter kernel.

## Setup
To generate the models, this notebook requires a clojupyter kernel with ipywidgets support. Compile and install the latest version [feature/ipywidgets](https://github.com/clojupyter/clojupyter/tree/feature/ipywidgets) branch.

To view and interact with the models you need to install the jupyter lab/notebook extensions for [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [ipywidgets](https://github.com/jupyter-widgets/ipywidgets).

In [ ]:
(ns notebook.leaflet
  (:require [clojupyter-plugin.leaflet :as leaf]
            [clojupyter-plugin.leaflet.colors :as colors]
            [clojupyter-plugin.widgets :as widget]
            [clojupyter-plugin.widgets.control :as inter]
            [cheshire.core :as json]
            [clojure.java.io :as io]
            [clojure.repl :refer :all]
            [clojure.pprint :refer [pprint]]));

Helper macro useful to attach widgets to global vars and show them in a single command.

In [ ]:
(defmacro def-show
  "Like def, but returns `init` if one was passed along."  
  ([symb]             `(def ~symb))
  ([symb init]        `(do (def ~symb ~init) ~symb))
  ([symb docstr init] `(do (def ~symb ~docstr ~init) ~symb)))

### Map Controls

In [ ]:
(def-show M0
  (let [sel (widget/dropdown :options (->> leaf/BASE-MAPS keys (map name) sort vec) :description "Base Map")
        b-maps (reduce merge (for [[k v] leaf/BASE-MAPS] {k (apply leaf/tile-layer (->> (update v :url format "2020-05-01") (reduce concat)))}))
        w-ctrl (leaf/widget-control :widget sel :position "topright") 
        dm (leaf/map :center [4 112] :zoom 3.6 :scroll_wheel_zoom true :basemap :open-topo-map
                     :controls [(leaf/scale-control :position "bottomleft") (leaf/full-screen-control) w-ctrl])]
    (inter/tie! #(conj (pop (:layers @dm)) (get b-maps (keyword %))) sel dm :value :layers)  
    dm))

In [ ]:
(def-show M1
  (let [l0 (apply leaf/tile-layer (reduce concat (apply list (:esri-world-imagery leaf/BASE-MAPS))))
        l1 (apply leaf/tile-layer (reduce concat (apply list (:nasagibs-viirs-earth-at-night-2012 leaf/BASE-MAPS))))
        sm (leaf/split-map-control :left_layer l0 :right_layer l1)]
    (leaf/map :controls [sm] :center [46, -35] :zoom 3 :basemap false :layout (widget/layout :height "60em"))))

In [ ]:
(let [dc (leaf/draw-control)
      mc (leaf/measure-control :primary_length_unit "kilometers" :primary_area_unit "hectares")]
  (leaf/map :center [38 27] :zoom 6 :controls [dc mc] :extra-controls? nil :basemap :carto-db-dark-matter))

In [ ]:
(leaf/map :center [43.12 41.99] :zoom 8
          :controls [(leaf/search-control :url "https://nominatim.openstreetmap.org/search?format=json&q={s}" :zoom 6)])

### Markers

Awesome-icon widget is not dynamic. You must refer to a new one to change it.

In [ ]:
(def-show M2
  (let [loc [52.205 360.121]
        ico (leaf/awesome-icon :name "address-card" :title ":boo")
        pop (widget/html :value "hello" :autoclose false)
        pin (leaf/marker :location loc :draggable true :rise_on_hover true :icon ico :title "None"
                         :popup pop)
        pin2 (leaf/marker :location [52.210 360.10] :draggable true :rise_on_hover true :icon ico :title "None"
                          :popup (widget/html :value "hi" :autoclose true :close_button false))
        lab (widget/label :value (apply (partial str "Pin at ") (:location @pin)))
        poppy (leaf/popup :child (widget/html :value "Click on a marker") :center [52.202399, 360.117888])
        #_mc #_(leaf/marker-cluster :markers [pin pin2])
        m (leaf/map :center loc :zoom 14 :layers [pin pin2 poppy])
        text (widget/text :description "Popup text" :continuous_update false)]
  (swap! pin assoc-in [:callbacks :on-move] (fn [_ {[lat lon] :location} _] (swap! lab assoc :value (format "Pin moved to %,4f, %,4f" lat lon))))   
  (inter/tie! identity text pop)
  (widget/v-box :children [text m lab])))

### Paths
#### Ant-Path

In [ ]:
(let [p (leaf/ant-path :locations [[51.185, 6.773], [51.182, 6.752], [51.185, 6.733], [51.194, 6.729],
                              [51.205, 6.732], [51.219, 6.723], [51.224, 6.723], [51.227, 6.728],
                              [51.228, 6.734], [51.226, 6.742], [51.221, 6.752], [51.221, 6.758],
                              [51.224, 6.765], [51.230, 6.768], [51.239, 6.765], [51.246, 6.758],
                              [51.252, 6.745], [51.257, 6.724], [51.262, 6.711], [51.271, 6.701],
                              [51.276, 6.702], [51.283, 6.710], [51.297, 6.725], [51.304, 6.732],
                              [51.312, 6.735], [51.320, 6.734], [51.326, 6.726], [51.334, 6.713],
                              [51.340, 6.696], [51.344, 6.678], [51.349, 6.662], [51.354, 6.655],
                              [51.360, 6.655], [51.366, 6.662], [51.369, 6.675], [51.373, 6.704],
                              [51.376, 6.715], [51.385, 6.732], [51.394, 6.741], [51.402, 6.743],
                              [51.411, 6.742], [51.420, 6.733], [51.429, 6.718], [51.439, 6.711],
                              [51.448, 6.716], [51.456, 6.724], [51.466, 6.719], [51.469, 6.713],
                              [51.470, 6.701], [51.473, 6.686], [51.479, 6.680], [51.484, 6.680],
                              [51.489, 6.685], [51.493, 6.700], [51.497, 6.714]],
                  :dash_array [1, 10],
                  :delay 1000,
                  :color "#7590ba",
                  :pulse_color "#3f6fba")]
  (leaf/map :center [51.332, 6.853] :zoom 10 :layers [p]))

In [ ]:
(def-show M4
  (let [but (repeatedly 4 #(widget/button :description "Reverse"))
        switch-bool {true false false true}        
        ap [(leaf/ant-path :locations [[51.5 8] [52.5 12.1]] :use "polyline" :popup (first but))
            (leaf/ant-path :locations [[52 8] [52.5 8] [52.3 9]] :use "polygon" :dash-array [50 100] :weight 10 :color "red" :popup (second but))
            (leaf/ant-path :locations [[51.5 11] [52 12]] :use "rectangle" :dash_array [10 20] :weight 5 :color "white" :pulse_color "green" :popup (nth but 2))            
            (leaf/ant-path :locations [51.5 10] :radius 15000 :use "circle" :dash_array [5 10] :weight 5 :color "magenta" :popup (last but))]]
   (doall (map #(swap! %1 assoc :callbacks {:on-click (fn [_ _ _] (swap! %2 update :reverse switch-bool))}) but ap))  
   (leaf/map :center [52 10] :zoom 8 :layers ap)))

#### Polylines

In [ ]:
(let [p0 (leaf/polyline :locations [[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]] :color "green" :fill false)
      p1 (leaf/polyline :locations [[[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]]
                              [[40.78 -73.91] [41.83 -87.62] [32.76 -96.72]]]
                        :color "green" :fill false)
      p2 (leaf/polygon :locations [[42 -60] [48 -34] [32 -48]] :color "green" :fill_color "green")
      p3 (leaf/polygon :locations [[[37 -109.05] [41 -109.03] [41 -102.05] [37 -102.04]]
                             [[37.29 -108.58] [40.71 -108.58] [40.71 -102.50] [37.29 -102.50]]]
                       :color "green" :fill_color "green")
      p4 (leaf/rectangle :bounds [[52 -6] [53 0]])
      p5 (leaf/circle :location [50 -6] :radius 300000 :color "green" :fill_color "green")
      p6 (leaf/circle-marker :location [55 0] :radius 50 :color "red" :fill_color "red")
      b-in (widget/toggle-buttons :options ["Polyline" "Multi-Polyline" "Polygon" "Multi-Polygon" "Rectangle" "Circle" "Circle-Marker"])
      paths [p0 p1 p2 p3 p4 p5 p6]
      switch-lay (fn [lays n-idx] (-> lays pop (conj (get paths n-idx))))
      m (leaf/map :center [42.5 -41] :zoom 3 :layers [p0])]
  (inter/tie! (partial switch-lay (:layers @m)) b-in m :index :layers)    
  (widget/v-box :children [b-in m]))

### Image / Video Overlay

In [ ]:
(let [v (leaf/video-overlay :url "https://www.mapbox.com/bites/00188/patricia_nasa.webm" :bounds [[13 -130] [32 -100]])
      m (leaf/map :center [23 -115] :zoom 4 :layers [v] :basemap :esri-world-imagery)]
  m)

---
## Geo Data

In [ ]:
(def GEO-DATA (json/parse-string (slurp "../data/NorthAmerica.geo.json")))
(def POP-MAP (reduce merge (map #(hash-map (get-in % ["properties" "formal_en"]) (get-in % ["properties" "pop_est"])) (get GEO-DATA "features"))));

Geo Data example uses pandas to load a dataset into memory. We need to identify its serial form to replicate on clojure side.

### Geo-JSON

In [ ]:
(def-show M3
  (let [lab (widget/label)
        h (fn [_ msg _] (swap! lab assoc :value (get-in msg [:properties :formal_en] "")))
        l (leaf/geo-json :data GEO-DATA :hover_style {:fillColor "red"} :callbacks {:on-mouseover h}
                         :style {:color "grey" :weight 1 :fillColor "lightyellow" :fillOpacity 0.7})
        m (leaf/map :center [40 -95] :zoom 3 :basemap :carto-db-positron :layers [l] :tooltip lab)]
  (widget/v-box :children [m lab])))

### Choropleth
Choropleth model is a geo-json with custom coloring based on some data.
Python model has external dependency for choosing the colours.

In [ ]:
(def-show M5
  (let [lab (widget/label)
        c (leaf/choropleth :data GEO-DATA :_data POP-MAP :key-path ["properties" "formal_en"]
                           :style {:fillOpacity 0.3 :dashArray "5, 5"}
                           :callbacks {:on-mouseover (fn [ref {{name :formal_en id :formal_en} :properties} _]
                                                       (swap! lab assoc :value (format "%s. Population: %,d" name (get (:_data @ref) id))))})]
    (leaf/map :layers [c] :zoom 3 :center [40 -80] :scroll_wheel_zoom true :basemap :carto-db-positron
              :controls [(leaf/widget-control :widget lab :position "topright")])))

In [ ]:
(def-show M6
  (let [v-max (reduce max (vals POP-MAP))
        v-min (reduce min (vals POP-MAP))
        col-fn (colors/single-hue-linear-fn v-min v-max "#D61717")
        lab (widget/label)
        c (leaf/choropleth :data GEO-DATA :_data POP-MAP :style {:fillOpacity 0.6 :dashArray "5, 5"}
                           :key-path ["properties" "formal_en"]
                           :color-fn (fn [m]
                                       (colors/single-hue-linear-fn (reduce min (vals m))
                                                                    (reduce max (vals m))
                                                                    "#D61717"))
                           :callbacks {:on-mouseover (fn [_ {{name :formal_en id :formal_en} :properties} _]
                                                       (swap! lab assoc :value (format "Population %s: %,d" name (get POP-MAP id))))})
      leg-k (take 8 (iterate (partial + (quot (- v-max v-min) 7)) v-min))
      leg (leaf/legend-control :position "bottomleft" :legend (zipmap (map (partial format "%,d") leg-k) (map col-fn leg-k)))
      m (leaf/map :layers [c] :center [45 -80] :zoom 3 :scroll_wheel_zoom true 
                  :controls [(leaf/full-screen-control) (leaf/widget-control :widget lab :position "topright") leg])]
  m))

### Heatmap

In [ ]:
(let [loc (vec (repeatedly 1000 #(vector (- (rand 160) 80) (- (rand 360) 180) (rand 1000))))
      hm (leaf/heatmap :locations loc :radius 20 :opacity 0.9)
      m (leaf/map :center [0 0] :zoom 2 :layers [hm] :controls [(leaf/scale-control :position "bottomleft")])]
 m)

### Not Tested
wms-layer, image-overlay, vector-tile-layer.
layer-group, feature-group

---
## Conclusions

### Layer Groups / Marker Clusters

Do we need them? Can we use a simple collection of layers/markers?

Yes, we need market cluster as is behaves differently on the front end.


----
## Debug

In [ ]:
#_(taoensso.timbre/set-level! :debug)